## Importing libraries
Python libraries are packages that have specific functions or tools that we want to use.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importing data and brief analysis
Here is where we're importing our data into our code so we can use it. We'll also look at the dataset itself to see what preprocessing we need to do.

**Goal: Depending on the time of call, service name, and council district, can we predict if the ticket is closed?**

In [2]:
reports = pd.read_csv('Citizen311Data_July_2018.csv')

reports.head()

,service_request_id,requested_datetime,updated_datetime,source,service_name,description,agency_responsible,address,council_district,status_description,lat,long
0,5088907,2018-07-01 01:38:00,NaN,UNKNOWN,ALLEY POTHOLE,"ALLEY POTHOLE, POTHL",LPM,"4305 W BROADWAY , LOUISVILLE, KY, 40211",5.0,OPEN,38.250791,-85.824421
1,5088879,2018-07-01 01:40:00,NaN,UNKNOWN,STREET TREE,"STREET EASEMENT TREES, OBSTRUCTION, ROWOB",LPM,"4305 W BROADWAY , LOUISVILLE, KY, 40211",5.0,OPEN,38.250791,-85.824421
2,5088846,2018-07-01 06:28:00,NaN,UNKNOWN,HIGH GRASS WEEDS,"HIGH GRASS, WEEDS, TREES, PRIVATE PROPERTY",IPL-PROPERTY MAINTENANCE DIV.,"7310 MARIA AVE , LOUISVILLE, KY, 40222",7.0,CLOSED,38.281501,-85.619034
3,5080683,2018-07-01 07:33:00,NaN,UNKNOWN,TRASH PVT PROP,"TRASH PRIVATE PROPERTY, TRASH, GARBAGE, JUNK",IPL-PROPERTY MAINTENANCE DIV.,"921 DENMARK ST , LOUISVILLE, KY, 40215",15.0,CLOSED,38.195922,-85.775900
4,5080684,2018-07-01 07:38:00,NaN,UNKNOWN,TRASH PVT PROP,"TRASH PRIVATE PROPERTY, TRASH, GARBAGE, JUNK",IPL-PROPERTY MAINTENANCE DIV.,"921 W EVELYN AVE , LOUISVILLE, KY, 40215",15.0,CLOSED,38.196846,-85.776010


In [3]:
print(reports.source.unique())
print(reports.updated_datetime.unique())

['UNKNOWN']
[nan]


We can drop (remove) the **updated_datetime** and **source** columns since there is only one value for all of the records. We can also drop the **service_request_id** column, as it has no bearing on whether a ticket is open or closed.

In [4]:
reports = reports.drop(columns=['updated_datetime', 'source', 'service_request_id'])

In [5]:
for x in np.sort(reports.agency_responsible.unique()):
    print(x)


4TH COUNCIL DISTRICT
AIR POLLUTION CONTROL
ANIMAL SERVICES
AT&T
CABLE COMPANY
COMMUNITY FORESTRY ARBORIST
EMERGENCY MANAGEMENT AGENCY
HEALTH DEPARTMENT
HEALTH DEPT - ENVIR. HEALTH
IPL - BUILDING DIVISION
IPL - PERMIT/LICENSE DIVISION
IPL - VACANT PROPERTY CLEANING
IPL-ELECTRICAL INSPECTION DIV.
IPL-INSPECTION,PERMIT,LICENSES
IPL-PROPERTY MAINTENANCE DIV.
KY DEPT OF TRANSPORTATION
LOUISVILLE FIRE DEPARTMENT
LOUISVILLE GAS AND ELECTRIC
LOUISVILLE WATER COMPANY
LPM
MAYOR'S OFFICE
METRO DEVELOPMENT AUTH.
METRO FACILITIES MANAGEMENT
METRO PARKS
METRO POLICE DEPARTMENT
METROCALL
METROPOLITAN SEWER DISTRICT
MSD DEVELOPMENT PLAN REVIEW
MSD FIELD ENGINEERING
MSD NON SERVICE CALL
MSD SANITARY MAINTENANCE
N-GOVERNMENT
N-METRO AGENCIES
N-MISCELLANEOUS
N-PICKUP DAYS
N-POLICE & JUDICIAL
N-SOCIAL SERVICES
N-UTILITY
N-WEATHER EVENTS
NON SERVICE SWMS
PARKING AUTHORITY OF RIVER CTY
PUBLIC WORKS - ELEC MAINT
PUBLIC WORKS - SIGNS MARKINGS
PUBLIC WORKS - TRAFFIC ENG
PUBLIC WORKS ADMINISTRATION
PUBLIC WORKS

In [6]:
for x in np.sort(reports.service_name.unique()):
    print(x)

ABANDON VEHICLE
ACCESS PARK ZONE
ALC BEV CONTROL
ALLEY DRAINAGE
ALLEY POTHOLE
ALLEY REPAIR
ALLEY RESURFACE
ALLEY TREE
ALLEY TREE EVAL
ANIMAL CRUELTY
ANIMAL NOISE
ANIMAL NUISANCE
APZ CURB RAMP
ASBESTOS
BASKETBALL GOAL
BEDBUGS CODES
BEDBUGS HEALTH
BIKE PEDESTRIAN
COMPLIMENT
CURB NEW
CURB REPAIR
DAMAGE GARB CART
DAMG RECYCL CART
DANGEROUS ANIMAL
DDI
DEAD ANIMAL
DUMPSTER OR POD
DUST
EASEMENT OTHER
EASEMNT DRAINAGE
ELECTRICAL
EMERGENCY MISC
EXTERIOR VIOL
FUMES
GARBAGE MISSED
GARBAGE VIOLATIO
GRAFFITI
GRAFFITI PVTPROP
GUARDRAIL NEW
GUARDRAIL REPAIR
HEALTH CONCERNS
HIGH GRASS WEEDS
ILLEGAL DUMPING
ILLEGAL PARKING
ILLEGAL SIGN
INTERIOR VIOL
JUNK MISSED
JUNK VIOLATION
LIGHTING NEW
LIGHTING REPAIR
LITTER BASKETS
LONG LINE STRIPE
LOUISVILLE LOOP
MARKING NEW
MARKING REFRESH
METRO OWNED PROP
MISCELLANEOUS
MOSQUITOS
NANIMAL SERVICES
NCAP NBHE PL MIN
NCODES & REGS
NCOUNCIL
NCOUNTY CLERK
NDEVLOU PAD BRTD
NEW BLDG ISSUE
NEW GARBAGE CART
NEW RECYCLE CART
NGARBAGE DAY
NGARBAGE QUEST
NHEALTH AND WELL
NHOM

In [7]:
print(len(reports.service_name.unique()))
print(len(reports.agency_responsible.unique()))

177
56


## Agency Acronym Guide
* IPL - Inspection, Permit, Licenses agency
* LPM - Louisville Pothole Management
* MSD - Metropolitan Sewer District
* SWMS - Sewer Waste Management Systems

### Back to cleaning :)
Some agencies repeat because they fall under different departments. We'll want to rename the departments under one agency instead.
Agencies with a "N-" in their name also need to be cleaned up, as well as "NON SERVICE SWMS"

In [8]:
agency_lst = pd.Series(reports['agency_responsible'].unique())
agency_lst[agency_lst.str.contains('HEALTH DEP')] # collect all values with the string 'HEALTH DEP'

13    HEALTH DEPT - ENVIR. HEALTH
39              HEALTH DEPARTMENT
dtype: object

In [9]:
reports['agency_responsible'] = reports.agency_responsible.replace(agency_lst[agency_lst.str.contains('HEALTH DEP')].tolist(), 'HEALTH DEPARTMENT')
reports['agency_responsible'] = reports.agency_responsible.replace(agency_lst[agency_lst.str.contains('IPL')].tolist(), 'IPL')

reports['agency_responsible'] = reports.agency_responsible.replace(agency_lst[agency_lst.str.contains('MSD')].tolist(), 'METROPOLITAN SEWER DISTRICT')

reports['agency_responsible'] = reports.agency_responsible.replace(agency_lst[agency_lst.str.contains('PUBLIC WORKS', case=False)].tolist(), 'PUBLIC WORKS')

reports['agency_responsible'] = reports.agency_responsible.replace(agency_lst[agency_lst.str.contains('SWMS', case=False)].tolist(), 'SWMS')

reports['agency_responsible'] = reports.agency_responsible.replace(regex=r'N-', value='')

reports['agency_responsible'] = reports.agency_responsible.replace(regex=r'NON SERVICE ', value='')


Checking if names are changed...

In [10]:
print(len(reports.agency_responsible.unique()))
for x in np.sort(reports.agency_responsible.unique()):
    print(x)

35
4TH COUNCIL DISTRICT
AIR POLLUTION CONTROL
ANIMAL SERVICES
AT&T
CABLE COMPANY
COMMUNITY FORESTRY ARBORIST
EMERGENCY MANAGEMENT AGENCY
GOVERNMENT
HEALTH DEPARTMENT
IPL
KY DEPT OF TRANSPORTATION
LOUISVILLE FIRE DEPARTMENT
LOUISVILLE GAS AND ELECTRIC
LOUISVILLE WATER COMPANY
LPM
MAYOR'S OFFICE
METRO AGENCIES
METRO DEVELOPMENT AUTH.
METRO FACILITIES MANAGEMENT
METRO PARKS
METRO POLICE DEPARTMENT
METROCALL
METROPOLITAN SEWER DISTRICT
MISCELLANEOUS
PARKING AUTHORITY OF RIVER CTY
PICKUP DAYS
POLICE & JUDICIAL
PUBLIC WORKS
SOCIAL SERVICES
ST. MATTHEWS (CITY OF)
SWMS
TRANSIT AUTH. OF RIVER CITY
UNABLE TO PROCESS
UTILITY
WEATHER EVENTS


## Dropping new columns
Dropping **address** since there's latitude and longitude as well as *council_district*.

Dropping **description** since it only serves to describe *service_name*.

In [11]:
reports = reports.drop(columns=['address', 'description'])
reports.head(1)

,requested_datetime,service_name,agency_responsible,council_district,status_description,lat,long
0,2018-07-01 01:38:00,ALLEY POTHOLE,LPM,5.0,OPEN,38.250791,-85.824421


## Datetimes
We want to get the day of the week and the time of day (morning, afternoon, night) from **requested_datetime**.

We'll have to convert the **requested_datetime** from a string to a datetime object.

### Day of week
Range from 0 (Monday) to 6 (Sunday)

### Time of day
* 0 - call was between midnight and 8am
* 1 - call was between 8am and 4pm
* 2 - call was between 4pm and midnight

In [12]:
reports['requested_datetime'] = pd.to_datetime(reports.requested_datetime)

In [13]:
reports['request_day_of_week'] = reports.requested_datetime.dt.dayofweek
reports['request_time_of_day'] = reports.requested_datetime.dt.hour

In [14]:
reports.loc[reports['request_time_of_day'] > 0, 'req_time_of_day'] = 0  #morning
reports.loc[reports['request_time_of_day'] > 8, 'req_time_of_day'] = 1 #afternoon
reports.loc[reports['request_time_of_day'] > 16, 'req_time_of_day'] = 2 #night

In [15]:
reports = reports.drop(columns=['requested_datetime', 'request_time_of_day'])

In [16]:
reports

,service_name,agency_responsible,council_district,status_description,lat,long,request_day_of_week,req_time_of_day
0,ALLEY POTHOLE,LPM,5.0,OPEN,38.250791,-85.824421,6,0.0
1,STREET TREE,LPM,5.0,OPEN,38.250791,-85.824421,6,0.0
2,HIGH GRASS WEEDS,IPL,7.0,CLOSED,38.281501,-85.619034,6,0.0
3,TRASH PVT PROP,IPL,15.0,CLOSED,38.195922,-85.775900,6,0.0
4,TRASH PVT PROP,IPL,15.0,CLOSED,38.196846,-85.776010,6,0.0
...,...,...,...,...,...,...,...,...
12381,ABANDON VEHICLE,METRO POLICE DEPARTMENT,14.0,CLOSED,38.099043,-85.871921,1,2.0
12382,HIGH GRASS WEEDS,IPL,16.0,CLOSED,38.300555,-85.638587,1,2.0
12383,STREET TREE,LPM,19.0,OPEN,38.249253,-85.460953,1,2.0
12384,EXTERIOR VIOL,IPL,5.0,CLOSED,38.264873,-85.780470,1,2.0


## Final edits: Encoding categorical values
We're encoding categorical data (string values) for use in our models.

### Types of Encoding
* **Label Encoding**: each value in the column corresponds to a number.
* **One Hot Encoding**: each value in the column corresponds to its own column. 1 denotes that the row has the value, 0 means it does not have that value.

*Editor's Note: One-hot encoding is a favorite of mine, as you don't have to worry as much about choosing algorithms that might assume a ranked category (unlike label encoding). However, it takes up more space in general and might not be as feasible for certain applications.*

### What needs encoding?
Checking the datatypes and from prior knowledge, we find these columns need encoding:
* *service_name*
* *agency_responsible*
* *council_district*
* *status_description*

### Service Name
There's 177 different values for service name, so we're going to be using label encoding for the purposes of this demonstration.

### Agency Responsible
Theres 35 agencies, so we could also use One-hot encoding for this column. However, we'll also be using label encoding.

### Council District
We're using label encoding. There's 27 districts, so it's not as large as service_name or agency_responsible.

### Status Description
We're using One Hot Encoding for demonstration. There's only 2 categories, so it's fairly easy to do.
* 0 - Closed
* 1 - Open

In [17]:
reports.dtypes

service_name            object
agency_responsible      object
council_district       float64
status_description      object
lat                    float64
long                   float64
request_day_of_week      int64
req_time_of_day        float64
dtype: object

In [18]:
print(len(reports.service_name.unique()))
print(len(reports.agency_responsible.unique()))
print(len(reports.council_district.unique()))
print(len(reports.status_description.unique()))

177
35
27
2


In [19]:
reports["agency_responsible"] = reports["agency_responsible"].astype('category')
reports["service_name"] = reports["service_name"].astype('category')
reports['council_district'] = reports['council_district'].astype('category')

In [20]:
#reports = pd.get_dummies(reports, columns=['status_description'])
reports['status_description'] = reports['status_description'].map({'OPEN':1, 'CLOSED':0})

In [21]:
reports.dtypes

service_name           category
agency_responsible     category
council_district       category
status_description        int64
lat                     float64
long                    float64
request_day_of_week       int64
req_time_of_day         float64
dtype: object

In [22]:
reports_oh = reports.copy()
reports['service_name'] = reports['service_name'].cat.codes
reports = pd.get_dummies(reports, columns=['council_district', 'agency_responsible'])
reports.head()

,service_name,status_description,lat,long,request_day_of_week,req_time_of_day,council_district_1.0,council_district_2.0,council_district_3.0,council_district_4.0,...,agency_responsible_PICKUP DAYS,agency_responsible_POLICE & JUDICIAL,agency_responsible_PUBLIC WORKS,agency_responsible_SOCIAL SERVICES,agency_responsible_ST. MATTHEWS (CITY OF),agency_responsible_SWMS,agency_responsible_TRANSIT AUTH. OF RIVER CITY,agency_responsible_UNABLE TO PROCESS,agency_responsible_UTILITY,agency_responsible_WEATHER EVENTS
0,4,1,38.250791,-85.824421,6,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,140,1,38.250791,-85.824421,6,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,41,0,38.281501,-85.619034,6,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,150,0,38.195922,-85.775900,6,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,150,0,38.196846,-85.776010,6,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#reports.to_csv('311data_cleaned.csv', index='False')

## Optional: All One Hot Encoding

In [24]:
reports_oh = pd.get_dummies(reports_oh, columns=['council_district', 'agency_responsible', 'service_name'])

In [25]:
reports_oh.head()

,status_description,lat,long,request_day_of_week,req_time_of_day,council_district_1.0,council_district_2.0,council_district_3.0,council_district_4.0,council_district_5.0,...,service_name_XANMC,service_name_XAVRW,service_name_XILPK,service_name_XMSC,service_name_XRWOB,service_name_XSTLT,service_name_XZONE,service_name_YARD WASTE MISS,service_name_YARD WASTE VIOL,service_name_ZONING CONCERNS
0,1,38.250791,-85.824421,6,0.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,38.250791,-85.824421,6,0.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,38.281501,-85.619034,6,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,38.195922,-85.775900,6,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,38.196846,-85.776010,6,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#reports_oh.to_csv('311data_cleaned_oh.csv', index='False')